In [2]:
import pandas as pd

In [5]:
df = pd.read_csv("/home/oscb/Mechanistic-Interpretation-Material-Inference/mimi/data/tr_data_type.csv")

In [6]:
df.columns

Index(['id', 'premise1', 'premise2', 'conclusion', 'type', 'new_type',
       'type_amr_op', 'Remarks (Ian)  '],
      dtype='object')

In [9]:
df

,id,premise1,premise2,conclusion,type,new_type,type_amr_op,Remarks (Ian)
0,0,climate means the average temperature / weathe...,winter has cold temperatures / weather,winter can cause the climate in an area to bec...,init,substitution,arg_substitution: (temperature/weather -> clim...,"invalid, but by equalizing similar notions, ye..."
1,1,drier means lower in amount of water / wetness...,the moon has less water / air than the earth,the moon is drier than the earth,substitution,NaN,frame_substitution: (has less water than -> is...,"yes, substitution, but also a Boolean (less wa..."
2,2,white fur is a kind of protection against pred...,snowy environments have a large amount of snow,white fur is a kind of protection against pred...,further_specification_or_conjunction,further_specification,arg_specification: (in the snow -> in a snowy ...,OK. But cf students read books -/-> students r...
3,3,a scar on the knee is a kind of scar,a scar is an acquired characteristic,a scar on the knee is an acquired characteristic,substitution,arg_substitution,arg_substitution: (scar -> an acquired charact...,"Yes. I would call it ""substitution in the pred..."
4,4,inventing paper allows paper to be used,if something is allowed to be used then the us...,inventing paper might increase the use of paper,inference_from_rule,NaN,conditional_frame_insertion,Isn't the conclusion part of the background kn...
...,...,...,...,...,...,...,...,...
4995,4995,photosynthesis is used to make food for plants,a plant requires photosynthesis to grow,plants require food to grow,substitution,NaN,NaN,NaN
4996,4996,panting and sweating are used to cool the body,an animal cooling itself is an adaptation to h...,an example of adaptations to hot temperatures ...,init,example,NaN,NaN
4997,4997,an example of camouflage is organism having th...,some environments change color,an example of camouflage is organism changing ...,init,substitution,NaN,NaN
4998,4998,northern hemisphere tilts towards the sun in s...,"when a hemisphere is tilted towards the sun , ...",northern hemisphere will receive more direct s...,inference_from_rule,NaN,NaN,NaN


In [ ]:
df[df["new_type"] == "substitution"] 

,id,premise1,premise2,conclusion,type,new_type,type_amr_op,Remarks (Ian)
0,0,climate means the average temperature / weathe...,winter has cold temperatures / weather,winter can cause the climate in an area to bec...,init,substitution,arg_substitution: (temperature/weather -> clim...,"invalid, but by equalizing similar notions, ye..."
8,8,the formation of diamonds requires intense pre...,the pressure is intense deep below earth 's crust,the formation of diamonds occurs deep below th...,init,substitution,frame_substitution: (requires -> form; intens...,Not sure why this isn't arg_substitution. But ...
11,11,mosquitos eat their food source,mosquitos eat blood of animals,animals are a source of food for mosquitos,init,substitution,frame_substitution: (animals -> food source),Why isn't this arg_substitution?
18,18,pollution has a negative impact on the environ...,runoff from a cropland causes water pollution,runoff from cropland has a negative impact on ...,init,substitution,arg_substitution: (pollution -> runoff from a ...,NaN
25,25,the nucleus of an atom is positve in electric ...,the nucleus is the center of an atom,the positive charge is concentrated in the cen...,init,substitution,frame_insertion,NaN
...,...,...,...,...,...,...,...,...
4821,4821,an example of a physical change is squeezing s...,"during digestion, food is squeezed through the...",an example of a physical change is squeezing f...,init,substitution,NaN,NaN
4853,4853,blowing air into a whistle causes air to vibra...,vibrations are a source of sound,the air vibrations inside the whistle are a so...,init,substitution,NaN,NaN
4993,4993,dogs will inherit inherited characteristics fr...,the color of / coloration of fur is an inherit...,dogs will have the same coloration of fur as t...,init,substitution,NaN,NaN
4997,4997,an example of camouflage is organism having th...,some environments change color,an example of camouflage is organism changing ...,init,substitution,NaN,NaN
